In [ ]:
# Instalación de dependencias
!pip install openai
!pip install python-dotenv
!pip install gradio
!pip install scrapegraphai
!pip install nest_asyncio
!pip install playwright
!pip install langchain langchain_openai
!pip install utils
!pip install azure-cognitiveservices-speech

In [ ]:
# Instalación de Playwright
!playwright install
!playwright install-deps

In [ ]:
import gradio as gr
from openai import AzureOpenAI
from dotenv import load_dotenv
import os
import nest_asyncio
from langchain_openai import AzureChatOpenAI
from typing import List
from scrapegraphai.graphs import SmartScraperGraph, SmartScraperMultiGraph
from pydantic import BaseModel, Field
import json
import string
import time
import threading
import wave
import utils
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

try:
    import azure.cognitiveservices.speech as speechsdk
except ImportError:
    print("""
    Importing the Speech SDK for Python failed.
    Refer to
    https://docs.microsoft.com/azure/cognitive-services/speech-service/quickstart-python for
    installation instructions.
    """)
    import sys
    sys.exit(1)

nest_asyncio.apply()

load_dotenv()

speech_key, service_region = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx", "westeurope"

# Configure Azure OpenAI API
client = AzureOpenAI(
    api_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
    api_version="2024-02-15-preview",
    azure_endpoint="https://openaicore-sweden.openai.azure.com/"
)

# Definir modelos Pydantic para el scraping
class ContactInfo(BaseModel):
    department: str = Field(description="El nombre del departamento")
    phone: str = Field(description="El número de teléfono del departamento")
    email: str = Field(description="La dirección de correo electrónico del departamento")

class ContactList(BaseModel):
    contacts: List[ContactInfo]

# Inicializar AzureChatOpenAI
llm = AzureChatOpenAI(deployment_name="gpt-4-turbo",
                      openai_api_version="2024-02-15-preview",
                      openai_api_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
                      azure_endpoint="https://openaicore-sweden.openai.azure.com")

model_tokens_count = 10000

graph_config = {
    "llm": {
        "model_instance": llm,
        "model_tokens": model_tokens_count,
        "temperature": 0,
    },
    "verbose": True,
    "loader_kwargs": {
        "slow_mo": 3000
    },
}

# Function to scrape contact information
def scrape_contact_info():
    smart_scraper_graph = SmartScraperGraph(
        prompt="Extract all department contact information including department name, phone number, and email address.",
        source="https://www.ual.es/universidad/serviciosgenerales/araties/contacto",
        config=graph_config,
        schema=ContactList
    )
    result = smart_scraper_graph.run()

    # Check if result is a dictionary and has a 'contacts' key
    if isinstance(result, dict) and 'contacts' in result:
        return result['contacts']
    else:
        # If the result doesn't have the expected structure, return an empty list
        print("Warning: Unexpected result structure from web scraping")
        return []

# Función para generar un resumen de la conversación
def generate_summary(history):
    if len(history) <= 3:
        return ""

    messages_to_summarize = [{"role": "system", "content": "Resume la siguiente conversación en un párrafo conciso:"}]
    for human, assistant in history[-3:]:
        messages_to_summarize.append({"role": "user", "content": human})
        messages_to_summarize.append({"role": "assistant", "content": assistant})

    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=messages_to_summarize,
        max_tokens=150,
        temperature=0.7
    )

    return response.choices[0].message.content

# Función para manejar las solicitudes del chatbot con memoria y resumen
def chatbot_with_memory_and_summary(message, history, contact_info):
    try:
        summary = generate_summary(history)

        # Formatear la información de contacto para incluirla en el mensaje del sistema
        if contact_info:
            contact_info_str = "\n".join([f"Departamento: {contact.get('department', '')}, Teléfono: {contact.get('phone', '')}, Email: {contact.get('email', '')}"
                                          for contact in contact_info])
        else:
            contact_info_str = "No se pudo obtener información de contacto en este momento."

        messages = [
            {"role": "system", "content": "Eres un asistente útil para la Universidad de Almería (UAL). Utiliza la siguiente información de contacto para responder consultas sobre los departamentos de la UAL:"},
            {"role": "system", "content": f"Informacion acerca Área de Gestión Académica y Apoyo al Estudiante: {contact_info_str}"},
            {"role": "system", "content": f"Resumen de la conversación: {summary}"}
        ]

        for human, assistant in history[-3:]:
            messages.append({"role": "user", "content": human})
            messages.append({"role": "assistant", "content": assistant})

        messages.append({"role": "user", "content": message})

        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=messages,
            max_tokens=500,
            temperature=0.7
        )

        bot_response = response.choices[0].message.content
        return bot_response
    except Exception as e:
        print(f"Error en chatbot_with_memory_and_summary: {str(e)}")
        return f"Lo siento, ha ocurrido un error: {str(e)}"

# Definir la interfaz de Gradio
def create_chat_interface():
    # Obtener la información de contacto mediante web scraping
    contact_info = scrape_contact_info()

    with gr.Blocks() as demo:
        gr.Markdown("## Chatbot UAL con Información de Contacto")

        chatbot = gr.Chatbot(label="Chat con Asistente UAL")
        msg = gr.Textbox(label="Tu mensaje")
        audio_input = gr.Audio(type="filepath", label="Graba tu mensaje")
        clear = gr.Button("Limpiar Chat")

        # Inicializar el historial como una lista vacía
        history = []

        def user_message(user_message, history):
            print(f"Mensaje recibido: {user_message}")
            print(f"Historial actual: {history}")
            try:
                bot_message = chatbot_with_memory_and_summary(user_message, history, contact_info)
                print(f"Respuesta del bot: {bot_message}")
                history.append((user_message, bot_message))
                return "", history
            except Exception as e:
                print(f"Error en user_message: {str(e)}")
                error_message = f"Lo siento, ha ocurrido un error: {str(e)}"
                history.append((user_message, error_message))
                return "", history

        msg.submit(user_message, [msg, chatbot], [msg, chatbot])
        audio_input.change(process_audio, audio_input, [msg, chatbot])
        clear.click(lambda: None, None, chatbot, queue=False)

    return demo

def process_audio(audio_input):
  """Realiza el reconocimiento de voz a partir de un archivo de audio"""
  if audio_input is None:
      return "No se recibió ningún archivo de audio.", []  # Mensaje vacío y lista vacía

  # Configuración del reconocimiento de voz
  speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
  audio_config = speechsdk.audio.AudioConfig(filename=audio_input)  # Asegúrate de usar el nombre del archivo
  speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, language="es-ES", audio_config=audio_config)

  # Reconocimiento de voz
  result = speech_recognizer.recognize_once()

  # Comprobar el resultado
  if result.reason == speechsdk.ResultReason.RecognizedSpeech:
      recognized_text = result.text
      print(f"Recognized: {recognized_text}")
      return recognized_text, []  # Retorna el texto reconocido y una lista vacía
  elif result.reason == speechsdk.ResultReason.NoMatch:
      print("No se pudo reconocer el habla.")
      return "No se pudo reconocer el habla.", []  # Mensaje de error
  elif result.reason == speechsdk.ResultReason.Canceled:
      cancellation_details = result.cancellation_details
      print(f"Speech Recognition canceled: {cancellation_details.reason}")
      if cancellation_details.reason == speechsdk.CancellationReason.Error:
          print(f"Error details: {cancellation_details.error_details}")
      return "Reconocimiento de voz cancelado.", []  # Mensaje de cancelación

# Ejecutar la interfaz
if __name__ == "__main__":
    demo = create_chat_interface()
    demo.launch()

ERROR:asyncio:Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/playwright/_impl/_connection.py", line 512, in wrap_api_call
    return await cb()
  File "/usr/local/lib/python3.10/dist-packages/playwright/_impl/_connection.py", line 85, in inner_send
    callback = self._connection._send_message_to_server(
  File "/usr/local/lib/python3.10/dist-packages/playwright/_impl/_connection.py", line 316, in _send_message_to_server
    raise self._closed_error
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
--- Executing Fetch Node ---
--- (Fetching HTML from: https://www.ual.es/universidad/serviciosgenerales/araties/contacto) ---
ERROR:web-loader:Attempt 1 failed: 
ERROR:asyncio:Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target pa

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://62e06d064452f7a0a3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
